In [1]:
#pip install pyodbc

In [2]:
#pip install openpyxl

In [3]:
#pip install sqlalchemy

In [4]:
# Emergency reload from github
#url = 'https://raw.githubusercontent.com/MarkMarquez0224/Resume-Portfolio/master/VC_News_Daily.csv'
# recovery_file = pd.read_csv(url)

In [5]:
#recovery_file.to_csv('VC_News_Daily.csv',index=False)

In [6]:
from bs4 import BeautifulSoup

import os

import pandas as pd

import pyodbc

import requests

import sqlalchemy
from sqlalchemy import create_engine

import time

In [7]:
os.chdir('C:\\Users\\marky\\OneDrive\\Documents\\Data Science\\Venture Funding and Companies\\')
os.getcwd()

'C:\\Users\\marky\\OneDrive\\Documents\\Data Science\\Venture Funding and Companies'

In [8]:
os.listdir()

['.ipynb_checkpoints',
 'VC News Daily - Web Scrap and Store.pdf',
 'VC News Web Scrap and Store.ipynb',
 'VC News Web Scrap and Store.ipynb - V2.ipynb',
 'VC_News_Daily-Test.csv',
 'VC_News_Daily.csv',
 'VC_News_Daily.xlsx',
 'VC_News_Daily_5_6.csv']

In [9]:
# Read website into 'BeautifulSoup' to help parse the website
website_soup = BeautifulSoup(requests.get('https://vcnewsdaily.com/').text)

In [10]:
# Read funding activity and store into list
funding_activity = website_soup.find_all('div',class_='col-md-8')

In [11]:
# For loop to read in 'funding_activity', get information(date, headline, website link), and store into list
funding_activity_list = [[funding_activity[i].small.text,funding_activity[i].h5.text,funding_activity[i].a.get('href')] for i in range(len(funding_activity))]
df_funding_activity_list = pd.DataFrame(funding_activity_list,columns=['Funding Date','Funding Title','Website Article Link'])
df_funding_activity_list.head()

,Funding Date,Funding Title,Website Article Link
0,28 May 2020,Stringr Announces $5.75M,https://vcnewsdaily.com/stringr/venture-capita...
1,28 May 2020,Caraway Scoops Up $5.3M Seed Financing,https://vcnewsdaily.com/caraway/venture-capita...
2,28 May 2020,Bright.md Closes $16.7 M Series C Round,https://vcnewsdaily.com/bright.md/venture-capi...
3,28 May 2020,Ascus Biosciences Closes $46M in Funding,https://vcnewsdaily.com/ascus-biosciences/vent...
4,28 May 2020,Benchling Lands $50M Series D Round,https://vcnewsdaily.com/benchling/venture-capi...


In [12]:
df_funding_activity_list.shape

(30, 3)

In [13]:
# Articles from VC News Daily put into series
website_article_links = df_funding_activity_list['Website Article Link']

In [14]:
# Articles from our CSV download put into a series
website_article_links_total = pd.read_csv('VC_News_Daily.csv')['Website Article Link']

In [15]:
# Checking for nbumber of links that aren't already saved
website_article_links_length = len(website_article_links[~website_article_links_total.isin(website_article_links)])
website_article_links_length

1

In [16]:
df_funding_activity_list = df_funding_activity_list[0:website_article_links_length]
df_funding_activity_list.head()

,Funding Date,Funding Title,Website Article Link
0,28 May 2020,Stringr Announces $5.75M,https://vcnewsdaily.com/stringr/venture-capita...


In [17]:
funding_activity_funding=[]
if website_article_links_length > 0:
    for i in range(len(df_funding_activity_list)):
        company_text = BeautifulSoup(requests.get(website_article_links[i]).text)
        funding_headline = company_text.find_all(id='summary')[0].text.strip('\n').strip('\t')
        if len(funding_headline.split(', ')) > 1:
            company_city = funding_headline.title().split(', ')[0]
            company_state = funding_headline.upper().split(', ')[1]
        else:
            company_city = 'N/A'
            company_state = 'N/A'
        website_company_link = company_text.find_all(id='fullArticle')[0].a.get('href')
        funding_article = company_text.find_all(id='fullArticle')[1].text
        funding_activity_funding.append([funding_headline,company_city,company_state,website_company_link,funding_article])

df_funding_activity_funding = pd.DataFrame(funding_activity_funding,columns=['Funding Headline','City','State','Website Company Link','Funding Article'])
df_funding_activity_funding.head()

,Funding Headline,City,State,Website Company Link,Funding Article
0,"NEW YORK, NY, Stringr has raised $5.75 million...",New York,NY,https://vcnewsdaily.com/Stringr/venture-fundin...,"Stringr, creator of the largest international ..."


In [18]:
df_funding_activity_funding.shape

(1, 5)

In [19]:
website_company_links = df_funding_activity_funding['Website Company Link']

In [20]:
funding_company_list = []
if website_article_links_length > 0:
    for i in range(len(df_funding_activity_list)):
        company_info = BeautifulSoup(requests.get(website_company_links[i]).text)
        company_name = company_info.find(class_='col-md-7 d-flex align-items-center').h5.text
        company_desc = company_info.find(class_='mt-5').p.text
        funding_type = company_info.find_all('td')[-2].text.strip('\n')
        funding_amount = company_info.find_all('td')[-3].text.strip('\n')
        funding_company_list.append([company_name,company_desc,funding_type,funding_amount])

df_funding_company_list = pd.DataFrame(funding_company_list,columns=['Company','Company Description','Latest Funding Round','Latest Funding Amount'])
df_funding_company_list.head()

,Company,Company Description,Latest Funding Round,Latest Funding Amount
0,Stringr,Stringr is a comprehensive video news and cont...,Venture,"$5,750,000"


In [21]:
df_funding_company_list.shape

(1, 4)

In [22]:
df_funding_data = pd.concat([df_funding_company_list,df_funding_activity_list,df_funding_activity_funding], axis=1)
df_funding_data.head()

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,Stringr,Stringr is a comprehensive video news and cont...,Venture,"$5,750,000",28 May 2020,Stringr Announces $5.75M,https://vcnewsdaily.com/stringr/venture-capita...,"NEW YORK, NY, Stringr has raised $5.75 million...",New York,NY,https://vcnewsdaily.com/Stringr/venture-fundin...,"Stringr, creator of the largest international ..."


In [23]:
df_funding_data.shape

(1, 12)

In [24]:
total_funding_data = pd.read_csv('VC_News_Daily.csv')

In [25]:
total_funding_data = pd.concat([df_funding_data,total_funding_data],axis=0).reset_index(drop=True)

In [26]:
# Temporary fix for duplicates (when needed) # total_funding_data = total_funding_data.iloc[18:].reset_index(drop=True)

In [27]:
total_funding_data.head(10)

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,Stringr,Stringr is a comprehensive video news and cont...,Venture,"$5,750,000",28 May 2020,Stringr Announces $5.75M,https://vcnewsdaily.com/stringr/venture-capita...,"NEW YORK, NY, Stringr has raised $5.75 million...",New York,NY,https://vcnewsdaily.com/Stringr/venture-fundin...,"Stringr, creator of the largest international ..."
1,Caraway,Caraway is on a mission to craft well-designed...,Seed,"$5,300,000",28 May 2020,Caraway Scoops Up $5.3M Seed Financing,https://vcnewsdaily.com/caraway/venture-capita...,"NEW YORK, NY, Caraway, the direct-to-consumer ...",New York,NY,https://vcnewsdaily.com/Caraway/venture-fundin...,Caraway is announcing today that it has raised...
2,Bright.md,Bright.md is transforming health care through ...,Series C,"$8,700,000",28 May 2020,Bright.md Closes $16.7 M Series C Round,https://vcnewsdaily.com/bright.md/venture-capi...,"PORTLAND, OR, Bright.md announced that it has ...",Portland,OR,https://vcnewsdaily.com/Bright.md/venture-fund...,"Bright.md, the leading healthcare automation c..."
3,Ascus Biosciences,Ascus uses its proprietary technology platform...,NaN,"$46,000,000",28 May 2020,Ascus Biosciences Closes $46M in Funding,https://vcnewsdaily.com/ascus-biosciences/vent...,"SAN DIEGO, CA, Ascus Biosciences has closed a ...",San Diego,CA,https://vcnewsdaily.com/Ascus%20Biosciences/ve...,The animal health and nutrition industry is ev...
4,Benchling,Benchling is the leader in cloud-based informa...,Series D,"$50,000,000",28 May 2020,Benchling Lands $50M Series D Round,https://vcnewsdaily.com/benchling/venture-capi...,"SAN FRANCISCO, CA, Benchling, the leading life...",San Francisco,CA,https://vcnewsdaily.com/Benchling/venture-fund...,The round was led by Alkeon and joined by new ...
5,Stackin',"Stackin', one of the fastest growing consumer ...",Series B,"$12,600,000",28 May 2020,Stackin' Scores $12.6M Series B Funding,https://vcnewsdaily.com/stackin%27/venture-cap...,"LOS ANGELES, CA, Stackin', a text message base...",Los Angeles,CA,https://vcnewsdaily.com/Stackin%27/venture-fun...,"The round was led by Octopus Ventures, along w..."
6,Rowdy Mermaid Kombucha,Rowdy Mermaid Kombucha combs the planet in sea...,Series A,"$3,500,000",28 May 2020,Rowdy Mermaid Kombucha Inks $3.5M Series A,https://vcnewsdaily.com/rowdy-mermaid-kombucha...,"BOULDER, CO, Rowdy Mermaid Kombucha has comple...",Boulder,CO,https://vcnewsdaily.com/Rowdy%20Mermaid%20Komb...,"Rowdy Mermaid Kombucha, a Boulder-based bevera..."
7,Synack,Synack has created the first system to safely ...,Series D,"$52,000,000",28 May 2020,Synack Raises $52M Series D Round,https://vcnewsdaily.com/synack/venture-capital...,"WASHINGTON, D.C., C5 Capital has led a success...",Washington,D.C.,https://vcnewsdaily.com/Synack/venture-funding...,"C5 Capital USA LLC (""C5 Capital""), the special..."
8,Stork Club,Stork Club is modernizing maternity care for t...,Seed,"$2,700,000",28 May 2020,Stork Club Pulls In $2.7M Seed Round,https://vcnewsdaily.com/stork-club/venture-cap...,"SAN FRANCISCO, CA, Stork Club has secured $2.7...",San Francisco,CA,https://vcnewsdaily.com/Stork%20Club/venture-f...,"Today, Stork Club unveils its comprehensive ma..."
9,Marqeta Payment,"Founded in 2010, Marqeta has re-engineered how...",Venture,"$150,000,000",28 May 2020,Marqeta Lands $150M,https://vcnewsdaily.com/marqeta-payment/ventur...,"OAKLAND, CA, Marqeta, the global modern card i...",Oakland,CA,https://vcnewsdaily.com/Marqeta%20Payment/vent...,Marqeta announced today that it has raised an ...


In [28]:
total_funding_data.shape

(151, 12)

In [29]:
total_funding_data.to_csv('VC_News_Daily.csv',index=False)

In [30]:
pd.read_csv('VC_News_Daily.csv').head(15)

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,Stringr,Stringr is a comprehensive video news and cont...,Venture,"$5,750,000",28 May 2020,Stringr Announces $5.75M,https://vcnewsdaily.com/stringr/venture-capita...,"NEW YORK, NY, Stringr has raised $5.75 million...",New York,NY,https://vcnewsdaily.com/Stringr/venture-fundin...,"Stringr, creator of the largest international ..."
1,Caraway,Caraway is on a mission to craft well-designed...,Seed,"$5,300,000",28 May 2020,Caraway Scoops Up $5.3M Seed Financing,https://vcnewsdaily.com/caraway/venture-capita...,"NEW YORK, NY, Caraway, the direct-to-consumer ...",New York,NY,https://vcnewsdaily.com/Caraway/venture-fundin...,Caraway is announcing today that it has raised...
2,Bright.md,Bright.md is transforming health care through ...,Series C,"$8,700,000",28 May 2020,Bright.md Closes $16.7 M Series C Round,https://vcnewsdaily.com/bright.md/venture-capi...,"PORTLAND, OR, Bright.md announced that it has ...",Portland,OR,https://vcnewsdaily.com/Bright.md/venture-fund...,"Bright.md, the leading healthcare automation c..."
3,Ascus Biosciences,Ascus uses its proprietary technology platform...,NaN,"$46,000,000",28 May 2020,Ascus Biosciences Closes $46M in Funding,https://vcnewsdaily.com/ascus-biosciences/vent...,"SAN DIEGO, CA, Ascus Biosciences has closed a ...",San Diego,CA,https://vcnewsdaily.com/Ascus%20Biosciences/ve...,The animal health and nutrition industry is ev...
4,Benchling,Benchling is the leader in cloud-based informa...,Series D,"$50,000,000",28 May 2020,Benchling Lands $50M Series D Round,https://vcnewsdaily.com/benchling/venture-capi...,"SAN FRANCISCO, CA, Benchling, the leading life...",San Francisco,CA,https://vcnewsdaily.com/Benchling/venture-fund...,The round was led by Alkeon and joined by new ...
5,Stackin',"Stackin', one of the fastest growing consumer ...",Series B,"$12,600,000",28 May 2020,Stackin' Scores $12.6M Series B Funding,https://vcnewsdaily.com/stackin%27/venture-cap...,"LOS ANGELES, CA, Stackin', a text message base...",Los Angeles,CA,https://vcnewsdaily.com/Stackin%27/venture-fun...,"The round was led by Octopus Ventures, along w..."
6,Rowdy Mermaid Kombucha,Rowdy Mermaid Kombucha combs the planet in sea...,Series A,"$3,500,000",28 May 2020,Rowdy Mermaid Kombucha Inks $3.5M Series A,https://vcnewsdaily.com/rowdy-mermaid-kombucha...,"BOULDER, CO, Rowdy Mermaid Kombucha has comple...",Boulder,CO,https://vcnewsdaily.com/Rowdy%20Mermaid%20Komb...,"Rowdy Mermaid Kombucha, a Boulder-based bevera..."
7,Synack,Synack has created the first system to safely ...,Series D,"$52,000,000",28 May 2020,Synack Raises $52M Series D Round,https://vcnewsdaily.com/synack/venture-capital...,"WASHINGTON, D.C., C5 Capital has led a success...",Washington,D.C.,https://vcnewsdaily.com/Synack/venture-funding...,"C5 Capital USA LLC (""C5 Capital""), the special..."
8,Stork Club,Stork Club is modernizing maternity care for t...,Seed,"$2,700,000",28 May 2020,Stork Club Pulls In $2.7M Seed Round,https://vcnewsdaily.com/stork-club/venture-cap...,"SAN FRANCISCO, CA, Stork Club has secured $2.7...",San Francisco,CA,https://vcnewsdaily.com/Stork%20Club/venture-f...,"Today, Stork Club unveils its comprehensive ma..."
9,Marqeta Payment,"Founded in 2010, Marqeta has re-engineered how...",Venture,"$150,000,000",28 May 2020,Marqeta Lands $150M,https://vcnewsdaily.com/marqeta-payment/ventur...,"OAKLAND, CA, Marqeta, the global modern card i...",Oakland,CA,https://vcnewsdaily.com/Marqeta%20Payment/vent...,Marqeta announced today that it has raised an ...


In [31]:
# Create excel file in order to upload into Microsoft SQL in order to upload dataset
total_funding_data.to_excel('VC_News_Daily.xlsx')

In [32]:
# Sensitive information withheld - via NULL
driver_name = '{ODBC Driver 17 for SQL Server}'
server_name = 'DESKTOP-OVRCKME'
database_name = 'Test'
trusted_connection = 'yes'

In [33]:
# Connect to SQL Server
conn = pyodbc.connect(Driver=driver_name,
                      Server=server_name,
                      Database=database_name,
                      Trusted_Connection=trusted_connection)
cursor = conn.cursor()

In [34]:
# Before adding new information - SQL Table as is, without the reordering
sql_raw_total_funding_data = pd.read_sql('SELECT * FROM VC_News_Daily',conn)
sql_raw_total_funding_data.head(15)

,index,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,0,Caraway,Caraway is on a mission to craft well-designed...,Seed,"$5,300,000",28 May 2020,Caraway Scoops Up $5.3M Seed Financing,https://vcnewsdaily.com/caraway/venture-capita...,"NEW YORK, NY, Caraway, the direct-to-consumer ...",New York,NY,https://vcnewsdaily.com/Caraway/venture-fundin...,Caraway is announcing today that it has raised...
1,10,Siren,Siren's vision is to empower people to avoid c...,Series B,"$11,800,000",28 May 2020,Siren Snares $11.8M in Series B,https://vcnewsdaily.com/siren/venture-capital-...,"SAN FRANCISCO, CA, Siren has raised an $11.8 m...",San Francisco,CA,https://vcnewsdaily.com/Siren/venture-funding.php,"According to TC, Siren has raised an $11.8 mil..."
2,20,Monte Rosa Therapeutics,Monte Rosa Therapeutics is a biotechnology com...,Series A,"$32,500,000",26 May 2020,Monte Rosa Therapeutics Inks $32.5M Series A,https://vcnewsdaily.com/monte-rosa-therapeutic...,"BOSTON, MA, Monte Rosa Therapeutics exited ste...",Boston,MA,https://vcnewsdaily.com/Monte%20Rosa%20Therape...,"Monte Rosa Therapeutics, a biotechnology compa..."
3,22,Seraphina Therapeutics,We're a health and wellness company dedicated ...,Series A,"$5,500,000",22 May 2020,Seraphina Therapeutics Raises $5.5M Series A,https://vcnewsdaily.com/seraphina-therapeutics...,"SAN DIEGO, CA, Seraphina Therapeutics has rais...",San Diego,CA,https://vcnewsdaily.com/Seraphina%20Therapeuti...,Seraphina Therapeutics has raised a $5.5M Seri...
4,1,Bright.md,Bright.md is transforming health care through ...,Series C,"$8,700,000",28 May 2020,Bright.md Closes $16.7 M Series C Round,https://vcnewsdaily.com/bright.md/venture-capi...,"PORTLAND, OR, Bright.md announced that it has ...",Portland,OR,https://vcnewsdaily.com/Bright.md/venture-fund...,"Bright.md, the leading healthcare automation c..."
5,2,Ascus Biosciences,Ascus uses its proprietary technology platform...,,"$46,000,000",28 May 2020,Ascus Biosciences Closes $46M in Funding,https://vcnewsdaily.com/ascus-biosciences/vent...,"SAN DIEGO, CA, Ascus Biosciences has closed a ...",San Diego,CA,https://vcnewsdaily.com/Ascus%20Biosciences/ve...,The animal health and nutrition industry is ev...
6,17,Enview,"Enview, Inc. automates geospatial big data for...",Venture,"$12,000,000",26 May 2020,Enview Announces $12M Oversubscribed Round,https://vcnewsdaily.com/enview/venture-capital...,"SAN FRANCISCO, , CA, Enview today announced th...",San Francisco,None,https://vcnewsdaily.com/Enview/venture-funding...,"Enview, a pioneer in the scalable processing o..."
7,3,Benchling,Benchling is the leader in cloud-based informa...,Series D,"$50,000,000",28 May 2020,Benchling Lands $50M Series D Round,https://vcnewsdaily.com/benchling/venture-capi...,"SAN FRANCISCO, CA, Benchling, the leading life...",San Francisco,CA,https://vcnewsdaily.com/Benchling/venture-fund...,The round was led by Alkeon and joined by new ...
8,23,Robocorp,Robocorp is set out to change Robotic Process ...,None,"$5,400,000",22 May 2020,Robocorp Collects $5.4M Round of Funding,https://vcnewsdaily.com/robocorp/venture-capit...,"SAN FRANCISCO, CA, Robocorp has closed a new $...",San Francisco,CA,https://vcnewsdaily.com/Robocorp/venture-fundi...,Robocorp has closed a new $5.4 million round o...
9,4,Stackin',"Stackin', one of the fastest growing consumer ...",Series B,"$12,600,000",28 May 2020,Stackin' Scores $12.6M Series B Funding,https://vcnewsdaily.com/stackin%27/venture-cap...,"LOS ANGELES, CA, Stackin', a text message base...",Los Angeles,CA,https://vcnewsdaily.com/Stackin%27/venture-fun...,"The round was led by Octopus Ventures, along w..."


In [35]:
# Sensitive information withheld
engine = sqlalchemy.create_engine('mssql+pyodbc://@DESKTOP-OVRCKME/Test?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')

In [36]:
total_funding_data.to_sql('VC_News_Daily',con=engine,if_exists='replace')

In [37]:
sql_total_funding_data = pd.read_sql('SELECT * FROM VC_News_Daily',conn).sort_values('index')
sql_total_funding_data.head(15)

,index,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,0,Stringr,Stringr is a comprehensive video news and cont...,Venture,"$5,750,000",28 May 2020,Stringr Announces $5.75M,https://vcnewsdaily.com/stringr/venture-capita...,"NEW YORK, NY, Stringr has raised $5.75 million...",New York,NY,https://vcnewsdaily.com/Stringr/venture-fundin...,"Stringr, creator of the largest international ..."
3,1,Caraway,Caraway is on a mission to craft well-designed...,Seed,"$5,300,000",28 May 2020,Caraway Scoops Up $5.3M Seed Financing,https://vcnewsdaily.com/caraway/venture-capita...,"NEW YORK, NY, Caraway, the direct-to-consumer ...",New York,NY,https://vcnewsdaily.com/Caraway/venture-fundin...,Caraway is announcing today that it has raised...
7,2,Bright.md,Bright.md is transforming health care through ...,Series C,"$8,700,000",28 May 2020,Bright.md Closes $16.7 M Series C Round,https://vcnewsdaily.com/bright.md/venture-capi...,"PORTLAND, OR, Bright.md announced that it has ...",Portland,OR,https://vcnewsdaily.com/Bright.md/venture-fund...,"Bright.md, the leading healthcare automation c..."
8,3,Ascus Biosciences,Ascus uses its proprietary technology platform...,None,"$46,000,000",28 May 2020,Ascus Biosciences Closes $46M in Funding,https://vcnewsdaily.com/ascus-biosciences/vent...,"SAN DIEGO, CA, Ascus Biosciences has closed a ...",San Diego,CA,https://vcnewsdaily.com/Ascus%20Biosciences/ve...,The animal health and nutrition industry is ev...
10,4,Benchling,Benchling is the leader in cloud-based informa...,Series D,"$50,000,000",28 May 2020,Benchling Lands $50M Series D Round,https://vcnewsdaily.com/benchling/venture-capi...,"SAN FRANCISCO, CA, Benchling, the leading life...",San Francisco,CA,https://vcnewsdaily.com/Benchling/venture-fund...,The round was led by Alkeon and joined by new ...
12,5,Stackin',"Stackin', one of the fastest growing consumer ...",Series B,"$12,600,000",28 May 2020,Stackin' Scores $12.6M Series B Funding,https://vcnewsdaily.com/stackin%27/venture-cap...,"LOS ANGELES, CA, Stackin', a text message base...",Los Angeles,CA,https://vcnewsdaily.com/Stackin%27/venture-fun...,"The round was led by Octopus Ventures, along w..."
13,6,Rowdy Mermaid Kombucha,Rowdy Mermaid Kombucha combs the planet in sea...,Series A,"$3,500,000",28 May 2020,Rowdy Mermaid Kombucha Inks $3.5M Series A,https://vcnewsdaily.com/rowdy-mermaid-kombucha...,"BOULDER, CO, Rowdy Mermaid Kombucha has comple...",Boulder,CO,https://vcnewsdaily.com/Rowdy%20Mermaid%20Komb...,"Rowdy Mermaid Kombucha, a Boulder-based bevera..."
14,7,Synack,Synack has created the first system to safely ...,Series D,"$52,000,000",28 May 2020,Synack Raises $52M Series D Round,https://vcnewsdaily.com/synack/venture-capital...,"WASHINGTON, D.C., C5 Capital has led a success...",Washington,D.C.,https://vcnewsdaily.com/Synack/venture-funding...,"C5 Capital USA LLC (""C5 Capital""), the special..."
15,8,Stork Club,Stork Club is modernizing maternity care for t...,Seed,"$2,700,000",28 May 2020,Stork Club Pulls In $2.7M Seed Round,https://vcnewsdaily.com/stork-club/venture-cap...,"SAN FRANCISCO, CA, Stork Club has secured $2.7...",San Francisco,CA,https://vcnewsdaily.com/Stork%20Club/venture-f...,"Today, Stork Club unveils its comprehensive ma..."
16,9,Marqeta Payment,"Founded in 2010, Marqeta has re-engineered how...",Venture,"$150,000,000",28 May 2020,Marqeta Lands $150M,https://vcnewsdaily.com/marqeta-payment/ventur...,"OAKLAND, CA, Marqeta, the global modern card i...",Oakland,CA,https://vcnewsdaily.com/Marqeta%20Payment/vent...,Marqeta announced today that it has raised an ...


In [38]:
sql_total_funding_data.shape

(151, 13)